In [1]:
import matplotlib.pyplot as plt
from keras.callbacks import Callback
from keras.applications import VGG16
from keras import models, optimizers, layers
from keras.preprocessing.image import ImageDataGenerator

C:\Users\Felipe\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class PlotLearning(Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.acc = []
        self.val_acc = []
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
        self.i += 1
        f, (ax1, ax2) = plt.subplots(1, 2, sharex=True)
        
        clear_output(wait=True)
        
        ax1.set_yscale('log')
        ax1.plot(self.x, self.losses, label="loss")
        ax1.plot(self.x, self.val_losses, label="val_loss")
        ax1.legend()
        
        ax2.plot(self.x, self.acc, label="accuracy")
        ax2.plot(self.x, self.val_acc, label="validation accuracy")
        ax2.legend()
        
        plt.show();
        
plot = PlotLearning()

In [3]:
height = 200
width = 250
batch_size = 100
train_dir = '../ImagesForFlow/train/'
test_dir = '../ImagesForFlow/test/'

In [4]:
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(height, width, 3))
vgg_conv.trainable = True
print(len(vgg_conv.trainable_weights))
set_trainable = False
for layer in vgg_conv.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
print(len(vgg_conv.trainable_weights))
vgg_conv.summary()

26
6
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200, 250, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 250, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 250, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 125, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 125, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 100, 125, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 50, 62, 128)       0         
_____

In [5]:
model = models.Sequential()

model.add(vgg_conv)

#model.add(layers.Flatten())
#model.add(layers.Dense(256, activation='relu'))
#model.add(layers.Dense(163, activation='sigmoid'))
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(163, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 6, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 21504)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              22021120  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 163)               167075    
Total params: 36,902,883
Trainable params: 29,267,619
Non-trainable params: 7,635,264
_________________________________________________________________


In [6]:
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()

val_batchsize = int(batch_size / 10)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    test_dir,
    target_size=(height, width),
    batch_size=val_batchsize,
    class_mode='categorical')

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

model.fit_generator(
    train_generator,
    steps_per_epoch=int(train_generator.samples / train_generator.batch_size),
    epochs=30,
    callbacks=[plot],
    validation_data=validation_generator,
    validation_steps=int(validation_generator.samples / validation_generator.batch_size),
    verbose=1)

Found 109325 images belonging to 163 classes.
Found 27340 images belonging to 163 classes.


AttributeError: module 'matplotlib' has no attribute 'figure'

In [ ]:
model.save('../Models/mark1.h5')